# GPU Computing - Exercise 3
## Eetu Knutars
## 29.1.2025

## Task 1
Implement Interleaved reduction kernel.

Importing libraries

In [1]:
import cupy as cp
import math
import numpy as np

Defining the kernel

In [2]:
# CUDA kernel
our_kernel = cp.RawKernel(r''' extern "C"
__global__ void interleaved_reduction(double* xs, int stride) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    // Check which indeces should be active for the current iteration
    if ((index & (2 *stride - 1)) == 0) {
        // Update the value for the left cell
        xs[index] += xs[index + stride];
    }
}
''', 'interleaved_reduction')

Generating data and calling the kernel

In [3]:
# Generate data
vector_size = 2**20
numThreadsPerBlock = 32
numBlocks = math.ceil(vector_size/numThreadsPerBlock)
num_iterations = int(math.log2(vector_size))

# Create input vector
data_type = np.float64
a = cp.random.randn(vector_size, dtype=data_type)
a_cpu = cp.asnumpy(a)

# Call the kernel
for i in range(0, num_iterations):
  # Stride is updated every iteration, the vector a is modified
  stride = 2**i
  our_kernel((numBlocks,1,1),(numThreadsPerBlock,1,1), (a, np.int32(stride)))

# Store the results and check if they match
result_gpu = cp.asnumpy(a)
result_cpu = np.sum(a_cpu)
if (np.allclose(result_gpu[0], result_cpu, 0.001, 0.001)):
  print("They are the same!")
else:
  print("Something must have gone wrong.:(")


They are the same!


## Task 2
Implement Interleaved reduction kernel with shared memory.

Importing libraries

In [4]:
import cupy as cp
import math
import numpy as np

Defining the kernel for interleaved reduction

In [5]:
our_kernel = cp.RawKernel(r''' extern "C"
#define SHARED_DIM 33
__global__ void interleaved_reduction(double *xs, double *result, int size)
{
    __shared__ double shared_xs[SHARED_DIM];
    int tid = threadIdx.x;
    int index = blockIdx.x * blockDim.x + tid;

    shared_xs[tid] = (index < size) ? xs[index] : 0;
    __syncthreads();

    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        if ((tid & (stride * 2 - 1)) == 0)
        shared_xs[tid] += shared_xs[tid + stride];
        __syncthreads();
    }

    if (tid == 0) result[blockIdx.x] = shared_xs[0];

}
''', 'interleaved_reduction')

Generate data and calculate the results

In [6]:
# Generate data
vector_size = 2**20
numThreadsPerBlock = 32
numBlocks = math.ceil(vector_size/numThreadsPerBlock)
num_iterations = int(math.log2(vector_size))

# Create input vector
data_type = np.float64
a = cp.random.randn(vector_size, dtype=data_type)
a_cpu = cp.asnumpy(a)

# Result vector
result = cp.zeros(numBlocks, dtype=data_type)

# Call the kernel
size = vector_size
for i in range(0, num_iterations):
  our_kernel((numBlocks,1,1),(numThreadsPerBlock,1,1), (a, result, np.int32(size)))
  size = math.ceil(size/numThreadsPerBlock)
  a = result

# Store the sums for GPU and CPU and check if they are same
result_gpu = cp.asnumpy(result)
result_cpu = np.sum(a_cpu)
if (np.allclose(result_gpu[0], result_cpu, 0.001, 0.001)):
  print("They are the same!")
else:
  print("Something must have gone wrong.:(")

They are the same!


## Task 3
Implement Sequential reduction with shared memory.

Importing libraries

In [7]:
import cupy as cp
import math
import numpy as np

Defining kernel

In [8]:
our_kernel = cp.RawKernel(r''' extern "C"
#define SHARED_DIM 33
__global__ void sequential_reduction(double *xs, double *result, int size)
{
    __shared__ double shared_xs[SHARED_DIM];
    int tid = threadIdx.x;
    int index = blockIdx.x * blockDim.x + tid;


    // Store data to shared memory
    shared_xs[tid] = (index < size) ? xs[index] : 0;
    __syncthreads();

    // Sequential reduction, update the stride in each iteration
    for (int stride = blockDim.x/2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            shared_xs[tid] += shared_xs[tid + stride];
        }
        __syncthreads();
    }

    // Store the value at index 0 to the result vector
    if (tid == 0) result[blockIdx.x] = shared_xs[0];

}
''', 'sequential_reduction')

Data generation and the kernel call for calculating the sum

In [9]:
# Generate data

vector_size = 2**20
numThreadsPerBlock = 32
numBlocks = math.ceil(vector_size/numThreadsPerBlock)
num_iterations = int(math.log2(vector_size))

# Create input vector
data_type = np.float64
a = cp.random.randn(vector_size, dtype=data_type)
a_cpu = cp.asnumpy(a)

# Vector for calculating the subresults
b = cp.zeros(numBlocks, dtype=data_type)

# Call the kernel
size = vector_size
for i in range(0, num_iterations):
  our_kernel((numBlocks,1,1),(numThreadsPerBlock,1,1), (a, b, np.int32(size)))
  size = math.ceil(size/numThreadsPerBlock)   # update the block size
  a = b                                       # update the input array

# Store the sums and check if they are the same
result_gpu = cp.asnumpy(b)
result_cpu = np.sum(a_cpu)
if (np.allclose(result_gpu[0], result_cpu, 0.001, 0.001)):
  print("They are the same!")
else:
  print("Something must have gone wrong.:(")

They are the same!
